# Analyze data

# 1. Imports

## 1.1 Packages

In [1]:
import pandas as pd

## 1.2 Options

## 1.3 Dataset

In [2]:
df_train = pd.read_csv("../data/01_raw/train.csv")
df_stores = pd.read_csv("../data/01_raw/stores.csv")
df_transactions = pd.read_csv("../data/01_raw/transactions.csv").sort_values(["store_nbr", "date"])

# 2. Prepare data

In [3]:
# Prepare datetime
df_train["date"] = pd.to_datetime(df_train.date)
df_transactions["date"] = pd.to_datetime(df_transactions.date)

In [4]:
df_train.sample(5)

,id,date,store_nbr,family,sales,onpromotion
1061973,1061973,2014-08-20,7,AUTOMOTIVE,1.0,0
1967004,1967004,2016-01-12,5,CELEBRATION,22.0,0
840594,840594,2014-04-18,44,HOME CARE,0.0,0
352023,352023,2013-07-17,36,GROCERY I,1725.0,0
923097,923097,2014-06-04,1,LINGERIE,32.0,0


In [5]:
df_transactions.sample(5)

,date,store_nbr,transactions
32685,2014-12-02,36,1265
60320,2016-06-01,37,1590
15226,2013-11-25,11,2088
41917,2015-06-11,54,703
50422,2015-11-23,45,3076
